## TP8 Neural Collaborative filtering

Nous allons dans ce TP prendre le dataset movielens "small". Dans un premier temps nous allons tester un modèle de factorisation de matrice avec Keras, ensuite, nous allons complexifier le modèle en ajoutant en sortie une couche de MLP (multilayer Layer Perceptron) afin comme vue en cours de complexifier le modèle et de pouvoir apprendre des relations non linéaires.

### I. Préparation des données

Le dataset étant déjà été analysé, nous allons ici simplement créer des dataframe et les diviser en un set d'entrainement et de validation.
- lisez tout d'abord le fichier ratings.csv avec pandas

Pour la suite de la préparation, nous allons nous inspire de l'exemple donné par la documentation Keras: https://keras.io/examples/structured_data/collaborative_filtering_movielens/

Vous allez encoder les identifiants des utilisateurs et des films, transformer les données pour qu'elles puissent être utilisées par un modèle de machine learning, et normaliser les notes. Suivez les étapes ci-dessous :


1. Créer des dictionnaires pour encoder les identifiants des utilisateurs et des films :
- Créez un dictionnaire user2user_encoded qui mappe chaque identifiant d'utilisateur unique (userId) à un indice séquentiel.
- Créez un dictionnaire inverse userencoded2user qui mappe chaque indice séquentiel à l'identifiant d'utilisateur correspondant.
- Créez un dictionnaire movie2movie_encoded qui mappe chaque identifiant de film unique (movieId) à un indice séquentiel.
- Créez un dictionnaire inverse movie_encoded2movie qui mappe chaque indice séquentiel à l'identifiant de film correspondant.

2. Transformer les colonnes userId et movieId en utilisant les dictionnaires d'encodage :
- Ajoutez une nouvelle colonne user au DataFrame df en mappant les valeurs de la colonne userId aux indices séquentiels en utilisant le dictionnaire user2user_encoded.
- Ajoutez une nouvelle colonne movie au DataFrame df en mappant les valeurs de la colonne movieId aux indices séquentiels en utilisant le dictionnaire movie2movie_encoded.

3. Calculer le nombre total d'utilisateurs et de films :
- Calculez le nombre total d'utilisateurs en mesurant la longueur du dictionnaire user2user_encoded et stockez-le dans la variable num_users.
- Calculez le nombre total de films en mesurant la longueur du dictionnaire movie_encoded2movie et stockez-le dans la variable num_movies.

4. Convertir les notes en type float32 :
- Convertissez les valeurs de la colonne rating en type float32 pour assurer la compatibilité avec le modèle TensorFlow/Keras.

5. Obtenir les notes minimales et maximales :
- Calculez la note minimale dans la colonne rating et stockez-la dans la variable min_rating.
- Calculez la note maximale dans la colonne rating et stockez-la dans la variable max_rating.
- Ces valeurs seront utilisées ultérieurement pour normaliser les notes lors de l'entraînement du modèle.

- Nous allons maintenant normaliser les données MinMaxScaler et diviser le dataset en x_train et x_val (ratio 90-10) en n'oubliant pas de mélanger. On pourra utiliser Sklearn

- Vous allez créer une classe RecommenderNet qui implémente un modèle de recommandation basé sur des embeddings pour les utilisateurs et les films. Ce modèle prédit les notes que les utilisateurs donneraient à des films qu'ils n'ont pas encore vus.
Étapes à suivre :

    - Initialisation du modèle :
        Créez un constructeur __init__ qui prend en paramètres le nombre d'utilisateurs (num_users), le nombre de films (num_movies) et la taille des embeddings (embedding_size).
        Dans ce constructeur, initialisez les couches d'embedding pour les utilisateurs et les films, ainsi que les biais correspondants. Assurez-vous d'utiliser une initialisation appropriée et d'appliquer une régularisation pour éviter le surapprentissage.

    - Définition de la méthode call :
        Implémentez la méthode call qui prend en entrée un tableau contenant les identifiants d'utilisateur et de film.
        Dans cette méthode, extrayez les vecteurs d'embedding et les biais pour l'utilisateur et le film.
        Calculez le produit scalaire entre le vecteur d'embedding de l'utilisateur et celui du film.
        Additionnez le produit scalaire aux biais de l'utilisateur et du film pour obtenir une prédiction de note.
        Appliquez une fonction d'activation sigmoïde à la sortie pour contraindre les prédictions entre 0 et 1.




Couche d'Embedding

Accès aux Vecteurs d'Embedding : Dans une couche d'embedding, chaque indice que vous passez est utilisé pour accéder à une ligne spécifique dans une matrice d'embedding. Chaque ligne de cette matrice représente un vecteur d'embedding pour une entité (par exemple, un film ou un utilisateur).

Pas de Multiplication par des Poids : Contrairement à la couche dense, la couche d'embedding ne multiplie pas directement les indices par des poids. Au lieu de cela, elle utilise les indices pour récupérer les vecteurs d'embedding correspondants. Ces vecteurs d'embedding sont des représentations denses qui ont été apprises pendant l'entraînement.
    Exemple : Si vous avez un indice 2, la couche d'embedding renverra le vecteur de la troisième ligne de la matrice d'embedding (puisque l'indexation commence à 0). Ce vecteur est déjà un ensemble de caractéristiques qui représente l'entité associée à cet indice.

Apprentissage des Vecteurs d'Embedding : Les vecteurs d'embedding eux-mêmes sont des poids qui sont appris pendant l'entraînement, mais ils ne sont pas multipliés par les indices lors de l'accès. Au lieu de cela, les indices servent simplement à sélectionner les vecteurs d'embedding appropriés.


Le modèle RecommenderNet que vous avez écrit est conceptuellement similaire à la factorisation de matrice. La factorisation de matrice consiste à décomposer une matrice de notes (où les lignes représentent les utilisateurs et les colonnes représentent les films) en deux matrices de rang inférieur : une matrice d'utilisateurs et une matrice de films.
    Dans le modèle RecommenderNet, les embeddings des utilisateurs et des films jouent un rôle similaire. Les vecteurs d'embedding pour les utilisateurs et les films capturent les caractéristiques latentes qui influencent les notes.

- Compilez maintenant le modèle avec le loss approprié et pour optimizer Adam (lr= 0.001). Comme métrique, on définiera "mean square error"


- Entrainez maintenant le modèle. Ce dernier a tendance a très vite overfitter, on va donc l'entraîner que sur 5 epochs. On pourra partir sur un batch size de 64 (selon le GPU)

- Ecrire une fonction plot_model_loss qui trace la courbe de loss du dataset d'entrainement et celle de la validation

- Evaluer le modèle sur le dataset d'évaluation: par exemple: " model.evaluate(x_val, y_val, verbose=1) " 

On observe comme nos données sont normalisées que la différence entre la prédiction et la vérité est de 4,5%

- Nous allons maintenant afficher quelques recommendation pour un utilisateur. Ecrire une fonction 'get_movie_recommendations' ayant 2 paramètres user_id et model. Cette fonction affichera les 10 recommandations les plus élevées pour un utilisateur donnée, avec le score et le genre des films. On affichera les 10 films les mieux notés par l'utilisateur. On s'attachera à vérifier plusieurs points:
    - La prédiction se fait avec l'id encodé. Mais l'affichage utilise l'id initial.
    - On ne fera des prédictions que sur des films non vus par l'utilisateur

## Partie II

Nous allons maintenant complexifier le modèle précedemment en ajoutant des couches de neurones dense en sortie afin de complexifier le modèle et de pouvoir apprendre des relations non-linéaires.
- Définissez une nouvelle classe DenseRecommenderNet qui reprend la même structure que RecommenderNet. En sortie de la multiplication des embeddings, ajouter 3 couches dense avec 128 neurones, 64 neurones, 32 neurones. Chaque couche est suivie d'une couche de Dropout (taux à 20%) afin de prévenir l'overfitting.

- Compilez et entrainez le modèle (vous pouvez modifier le learning rate et le nombre d'epoch) par rapport au premier modèle.

- Evaluez votre modèle, et afficher quelques recommandations avec ce nouveau modèle